### Проверка работоспособности и качества пайплайна

Здесь мы еще не запускаем никакое API, а загружаем модель (pipeline) напрямую и проверяем на отложенной (тестовой) выборке

In [13]:
!pip install catboost

     |████████████████████████████████| 76.3 MB 1.2 MB/s 


In [14]:
import pandas as pd
from sklearn.metrics import roc_auc_score,roc_curve #,scorer
from catboost import CatBoostClassifier
import dill
dill._dill._reverse_typemap['ClassType'] = type

In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [8]:
DATA_PATH = '/gdrive/My Drive/Ml_In_Business/kursovaya/'
BUY_SFX = '_BUY'
DATA_PATH

'/gdrive/My Drive/Ml_In_Business/kursovaya/'

In [9]:
X_test = pd.read_csv(f"{DATA_PATH}X_test{BUY_SFX}.csv")
y_test = pd.read_csv(f"{DATA_PATH}y_test{BUY_SFX}.csv")

In [ ]:
X_test.head(3)

,open,high,low,close,EMA_4_ema4o,EMA_5_ema5c,spread,tick_volume,CCI_14_0.015,RSI_14,MOM_14,STOCHRSIk_14_14_3_3,STOCHRSId_14_14_3_3
0,1240.09,1240.09,1231.85,1233.86,1238.877201,1236.877101,160,7587,2.307331,49.062874,1.92,5.491790e+01,55.584798
1,1797.14,1798.04,1795.24,1796.81,1801.515545,1800.450619,160,5411,-231.135996,30.133513,-14.29,1.231607e-13,17.777861
2,1864.21,1866.99,1860.72,1864.00,1865.316173,1865.301380,140,11151,-144.385668,48.745362,-4.92,1.187123e+01,8.593983


In [ ]:
# with open('logreg_pipeline.dill', 'rb') as in_strm:
#     pipeline = dill.load(in_strm)


In [17]:
with open(f"{DATA_PATH}catboost_pipeline.dill", 'rb') as in_strm:
    dmodel = dill.load(in_strm)

In [18]:
dmodel

{'date': '09_01_2022_181020', 'model': Pipeline(steps=[('preprocessor',
                  ColumnTransformer(transformers=[('num_price',
                                                   Pipeline(steps=[('imputer',
                                                                    SimpleImputer(strategy='median')),
                                                                   ('scaler',
                                                                    RobustScaler())]),
                                                   ['open', 'high', 'low',
                                                    'close', 'EMA_4_ema4o',
                                                    'EMA_5_ema5c', 'spread',
                                                    'tick_volume',
                                                    'CCI_14_0.015']),
                                                  ('num_ind',
                                                   Pipeline(steps=[('imputer',
           

In [19]:
pipeline = dmodel.get("model")

In [20]:
predictions = pipeline.predict_proba(X_test)
pd.DataFrame({'preds': predictions[:, 1]}).to_csv(f"{DATA_PATH}test_predictions{BUY_SFX}.csv", index=None)

In [21]:
roc_auc_score(y_score=predictions[:, 1][:], y_true=y_test.iloc[:])

0.692023441493775